<a href="https://colab.research.google.com/github/YusunPark/BigData/blob/main/%EC%A0%9C1%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 단순한 데이터 분석

- data.sort_values(by= , ascending=)
- data.fillna(대체할 값, inplace=True)
- data.dropna(inplace=True) : 결측치 행 삭제
- new_data = data.drop(columns = ['CHAR', 'RAD'])
- new_data.iloc[0:29] = 41.7 (값 대체)
- data.groupby(['AGE'])['AGE'].count()

In [1]:
import pandas as pd
import numpy as np

boston = pd.read_csv('/content/drive/MyDrive/빅분기 실기/boston.csv')

In [ ]:
# 1.1 Top 10 구하기
# MEDV 컬럼 오름차순 정렬된 값의 상위 10개  -> sort_value(by= , ascending=)

import pandas as pd
boston = pd.read_csv('/content/drive/MyDrive/빅분기 실기/boston.csv')

print(boston.sort_values(by='MEDV')['MEDV'].head(10))

398    5.0
405    5.0
400    5.6
399    6.3
414    7.0
489    7.0
401    7.2
385    7.2
415    7.2
387    7.4
Name: MEDV, dtype: float64


In [ ]:
# 1.2 결측치 확인하기
# boston의 RM컬럼에 대한 결측치를 
# -> 평균값 대치후 산출된 표준편치 값 - 삭제후 산출된 표준편차의 값 차이

boston['RM'].isnull().sum()

15

In [ ]:
mean_rm = boston['RM'].copy()
mean_rm.fillna(mean_rm.mean(), inplace=True)
mean_rm.describe()

count    506.000000
mean       6.285102
std        0.697500
min        3.561000
25%        5.889500
50%        6.224500
75%        6.605500
max        8.780000
Name: RM, dtype: float64

In [ ]:
del_rm = boston['RM'].copy()
del_rm = del_rm.dropna()
del_rm.describe()

count    491.000000
mean       6.285102
std        0.708096
min        3.561000
25%        5.886000
50%        6.209000
75%        6.622000
max        8.780000
Name: RM, dtype: float64

In [ ]:
print(abs(mean_rm.std() - del_rm.std()))

0.010595546094104513


In [ ]:
# 1.3 이상값 확인하기
# boston에서 ZN 칼럼의 평균값에서 표전편차의 1.5배보다 크거나 작은 ZN값의 합계

zn_max = boston['ZN'].mean() + (1.5 * boston['ZN'].std())
zn_min = boston['ZN'].mean() - (1.5 * boston['ZN'].std())

max = boston[boston['ZN'] > zn_max]['ZN'].sum()
min = boston[boston['ZN'] < zn_min]['ZN'].sum()

print(max+min)

3462.5


In [ ]:
# 1.4 사분위수 구하기
# boston 데이터에서 CHAS, RAD칼럼을 제외한 칼럼에 한해서 IQR 값을 구해라
# 출력구조는 2열이고 , 1열에는 컬럼의 이름

data = boston.drop(columns = ['CHAS', 'RAD'])
data_desc = data.describe().transpose()

print(data_desc['75%'] - data_desc['25%'])

CRIM         3.595038
ZN          12.500000
INDUS       12.910000
NOX          0.175000
RM           0.736000
AGE         49.050000
DIS          3.088250
TAX        387.000000
PTRATIO      2.800000
B           20.847500
LSTAT       10.005000
MEDV         7.975000
dtype: float64


In [ ]:
# 1.5 순위 구하기
# boston 데이터에서 MEDV칼럼을 기준으로 30번째 큰 값을, 1~29번째로 큰 값에 적용. 
# -> 그 후 칼럼의 평균, 중위, 최소, 최대값을 한 줄로

data_medv = boston['MEDV'].copy()
new_data = data_medv.sort_values(ascending=False)
new_data.iloc[0:29] = 41.7
print(new_data.mean(), new_data.median(), new_data.min(), new_data.max())

22.1796442687747 21.2 5.0 41.7


## 2. 복잡한 데이터 분석

In [ ]:
# 2.1 그룹별 집계 / 요약하기
# boston 데이터의 TAX 칼럼에서 중위값보다 큰 데이터 대상으로 CHAS, RAD 칼럼 순으로 그룹을 지은 후
# 각 그룹의 데이터의 개수를 구하라

median = boston['TAX'].median()
new_data = boston[boston['TAX'] > median][['CHAS', 'RAD']]
group_data = new_data.groupby(by=['CHAS', 'RAD'])['RAD'].count()
# print(type(group_data))
group_data = pd.DataFrame(group_data)
group_data.columns = ['COUNT']
print(group_data)

          COUNT
CHAS RAD       
0    1        3
     2        2
     3        5
     4       33
     5       51
     6       17
     24     124
1    5        7
     24       8


In [ ]:
# 2.2 오름차순/내림차순 정렬하기
# boston 데이터의 TAX컬럼을 오름차순/내림차순으로 정렬하고, 각 쌍의 차이의 분산을 구하라

ascending = boston['TAX'].sort_values().reset_index(drop=True)
descending = boston['TAX'].sort_values(ascending=False).reset_index(drop=True)

diff = abs(ascending - descending)
diff.describe()

# 분산은 var/ 표준편차는 std
print(diff.var())

28490.598645951555


In [8]:
# 2.3 최소최대 변환하기
# boston 데이터의 MEDV컬럼을 MinMaxScale로 변환하고 0.5보다 큰 값을 가지는 레코드의 수를 구하라

from sklearn.preprocessing import MinMaxScaler

medv = boston.copy()

scaler = MinMaxScaler()
data_minmax = scaler.fit_transform(medv)

# data_minmax 변수를 데이터 프레임으로 변환하기
data_minmax = pd.DataFrame(data_minmax, columns = medv.columns)

print(data_minmax[data_minmax['MEDV'] > 0.5]['MEDV'].count())

106


In [52]:
# 2.4 빈도값 구하기
# boston 데이터의 AGE 칼럼을 소수점 첫 번째 자리에서 반올림하고,
# 가장 많은 비중을 차지하는 AGE 값과 그 개수를 차례대로 출력하라 -> 최빈값과, 그의 개수 출력

age = boston['AGE']
age = age.round(0)
data = pd.DataFrame(age)
data2 = pd.DataFrame(data.groupby(['AGE'])['AGE'].count())
data2.columns = ['COUNT']

data2.reset_index(drop=False, inplace=True)
data2.sort_values(by='COUNT', ascending=False, inplace=True)

print(data2.iloc[0][1])

43.0


In [53]:
# 2.4 빈도값 구하기
# boston 데이터의 AGE 칼럼을 소수점 첫 번째 자리에서 반올림하고,
# 가장 많은 비중을 차지하는 AGE 값과 그 개수를 차례대로 출력하라 -> 최빈값과, 그의 개수 출력

from scipy.stats import mode 

age = pd.DataFrame(boston['AGE'].round(0))
print(mode(age))


ModeResult(mode=array([[100.]]), count=array([[43]]))


In [63]:
# 2.5 표준 변환하기 (StandardScaler)
# boston 데이터의 DIS컬럼을 표준화 척도로 변환후, 0.4~0.6 값에 대한 평균을 구하라

from sklearn.preprocessing import StandardScaler

data = boston.copy()
scaler = StandardScaler()

scale_data = scaler.fit_transform(data)
data2 = pd.DataFrame(scale_data, columns=data.columns)

print(data2[(data2['DIS'] < 0.6) & (data2['DIS']> 0.4) ]['DIS'].mean().round(3))

0.48

In [64]:
# 2.6 유니크한 값 구하기
# boston 데이터의 전체 컬럼에서 중복을 제거한 유니크한 값을 구하고, 컬럼별로 유니크한 값의 개수를 기준으로 평균값

data = boston.copy()


In [72]:
sum = 0
count = 0
for i in data:
  sum += len(data[i].unique())
  count += 1

print(sum / count)

  

218.07142857142858


In [73]:
sum

3053

In [75]:
sum = 0

for i in data:
  sum += int(pd.DataFrame(data[i].unique()).count())

print(sum)

  

3052
